In [1]:
import pandas as pd

import glob

from pathlib import Path
import csv
import pickle

# import os
# import requests
# import json

# import quandl

# from dotenv import load_dotenv
# load_dotenv()

In [2]:
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [3]:
pkl_path_1 = Path('../Resources/complete_data_0_10000.pkl')
pkl_path_2 = Path('../Resources/incomplete_data_0_10000.pkl')
complete_data  = load_obj(pkl_path_1)
incomplete_data= load_obj(pkl_path_2)

In [4]:
import yfinance as yf

def get_ticker_info_yf(symbol):
    str_symbol = str(symbol)
    ticker = yf.Ticker(str_symbol)
    try:
        ticker_info = ticker.info
    except TypeError:
        ticker_info = {
            'sharesOutstanding' : 0,
            'floatShares': 0
        }
    try:
        sharesOutstanding = ticker_info['sharesOutstanding']
    except KeyError:
        sharesOutstanding = 0
    try:
        floatShares = ticker_info['floatShares']
    except KeyError:
        floatShares = 0
    dictionary = {
        'sharesOutstanding' : sharesOutstanding,
        'floatShares' : floatShares
    }
    return dictionary

In [5]:
key_list = []
for key in complete_data.keys():
    key_list.append(key)
length_ = len(key_list)
length_

2519

In [8]:
## Iterate through in blocks of 100 

## Start at 0 / 100 , increase by 100 

x = 2500
y = 2519
increment = 100
#select_symbol_list = []

while (x < length_):
    info_dict = {}
    selected_dict = {}


    range_var = range(x,y)
    str_symbol1 = str(x)
    str_symbol2 = str(y)


    for counter in range_var:
        i = key_list[counter]   ## i will equal symbol for key 
        df = complete_data[i].copy()
        ftd_sum = df.QUANTITY_FAILS.sum()
        i_info = get_ticker_info_yf(i)
        try:
            if i_info['sharesOutstanding'] != 0:
                pct_ftd_SO = ftd_sum / i_info['sharesOutstanding']
            else:
                pct_ftd_SO = 0
            if i_info['floatShares'] != 0:
                pct_ftd_float = ftd_sum / i_info['floatShares']
            else:
                pct_ftd_float = 0
        except TypeError:
            pct_ftd_SO = 0
            pct_ftd_float = 0

        info_dict[i] = {
            'FTD_sum' : ftd_sum,
            'sharesOutstanding' : i_info['sharesOutstanding'],
            'floatShares' : i_info['floatShares'],
            'pct_ftd_SO' : pct_ftd_SO,
            'pct_ftd_float' : pct_ftd_float
        }

        if pct_ftd_float > 1.0:
            select_symbol_list.append(i)
            selected_dict[i] = {
            'FTD_sum' : ftd_sum,
            'sharesOutstanding' : i_info['sharesOutstanding'],
            'floatShares' : i_info['floatShares'],
            'pct_ftd_SO' : pct_ftd_SO,
            'pct_ftd_float' : pct_ftd_float
            }

    ## Export if it finishes
    yf_export_path = Path('../FilesExportYF_Data/yf_data_'+str_symbol1+'_'+str_symbol2+'.pkl')
    save_obj(info_dict,yf_export_path)
    select_export_path = Path('../FilesExportSelectYF_Data/select_yf_data_'+str_symbol1+'_'+str_symbol2+'.pkl')
    save_obj(selected_dict,select_export_path)
    
    x += increment
    y += increment
    
## Export List at End 
select_list_path = Path('../Resources/select_symbol_list.pkl')
save_obj(select_symbol_list,select_list_path)

In [7]:
select_list_path = Path('../Resources/select_symbol_list.pkl')
save_obj(select_symbol_list,select_list_path)